# Dynamic pricing

* A pricing strategy that adjusts the price of a product or service in real-time
* Based on factors:
  * Market demand
  * Competition
  * Customer behaviour
* Goal is to maximize profits 
  * By finding the optimal price point that customers are willing to pay
  * Costs associated with producing and selling the product


## Algorithms

### Rule-based
Uses pre-defined rules to set prices i.e. increase price if demand is high or lower price if demand is low.

### Time-series forecasting
Uses historical data to forecast future demand and adjust prices accordingly. Takes into account seasonal patterns and trends to optimize pricing.

### Machine Learning algorithm
Use historical data to learn patterns and adjust prices based on predictive models. Can include regression analysis, decision trees, and neural networks.

### Reinforcement Learning algorithm
Uses trial and error to optimize pricing. Algorithm makes decisions based on feedback from customer behaviour and ajusts prices to maximize profit.

### Multi-armed bandit algorithm
Used in situations where there are multiple products or pricing options. Uses trial and error to test different pricing strategies and learns which ones are most effective in maximizing profit.


In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

### Join datasets

* Brazilian E-commerce data in multiple CSVs

In [5]:
# Read multiple CSV files
FILE_PATH = "/workspaces/dynamic_pricing/data/"

order_items    = pd.read_csv(FILE_PATH + "olist_order_items_dataset.csv")
orders         = pd.read_csv(FILE_PATH + "olist_orders_dataset.csv")
order_payments = pd.read_csv(FILE_PATH + "olist_order_payments_dataset.csv")
products       = pd.read_csv(FILE_PATH + "olist_products_dataset.csv")
customers      = pd.read_csv(FILE_PATH + "olist_customers_dataset.csv")
sellers        = pd.read_csv(FILE_PATH + "olist_sellers_dataset.csv")
product_category_translation = pd.read_csv(FILE_PATH + "product_category_name_translation.csv")

# Merge datasets
merged = order_items.merge(orders, on='order_id') \
                    .merge(order_payments, on=['order_id']) \
                    .merge(products, on='product_id') \
                    .merge(customers, on='customer_id') \
                    .merge(sellers, on='seller_id') \
                    .merge(product_category_translation, on='product_category_name')

# Save the consolidated dataset to a CSV file
merged.to_csv(FILE_PATH + 'brazilian_ecommerce_dataset.csv', index=False)